<a href="https://colab.research.google.com/github/abubakarzohaib141/Chatbot1415/blob/main/crewai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install google-generativeai
!pip install langchain_community  # Ensure this is installed

import os
from IPython import get_ipython
from IPython.display import display
# %%
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
# %%
# Warning control
import warnings
warnings.filterwarnings('ignore')
# %%
from crewai import Agent, Task, Crew
# %%
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import GooglePalm
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")
def get_gemini_api_key():
  """get gemini api key"""
  # This function is expected to return the Gemini API key.
  # Replace 'YOUR_GEMINI_API_KEY' with your actual key.
  return gemini_api_key  # Replace with your actual Gemini API key

gemini_api_key = get_gemini_api_key()
# Setting the Gemini model name
os.environ["GEMINI_MODEL_NAME"] = 'gemini-1.5-flash'  # or 'gemini-pro'
# Setting the Google AI API key
os.environ["GOOGLE_API_KEY"] = gemini_api_key

# Override the ChatOpenAI class with GooglePalm to use Gemini
class ChatOpenAI(GooglePalm):
    def __init__(self, *args, **kwargs):
        # Force the GooglePalm model to use Gemini
        kwargs['model_name'] = os.environ.get('GEMINI_MODEL_NAME', 'gemini-pro')
        super().__init__(*args, **kwargs)


# Define Agents
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(),  # Use the modified ChatOpenAI for Gemini
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(),  # Use the modified ChatOpenAI for Gemini
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(),  # Use the modified ChatOpenAI for Gemini
)

# Define Tasks
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

# Create Crew
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

# Kickoff the Crew
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

  Using cached langchain_community-0.3.15-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.31-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_community-0.3.15-py3-none-any.whl (2.5 MB)
Using cached langchain_core-0.3.31-py3-none-any.whl (412 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires langchain-core<0.3,>=0.1.46, but you have langchain-core 0.3.31 which is incompatible.
  Using cached crew